In [27]:
import itertools as it
import networkx as nx   # necessary for network structure plots
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


# Loading data from a text file

The following cell loads the data stored in the text files "train.txt" and "test.txt". This results in two NumPy arrays with shapes 500x5 (train.txt) and 5000x5 (test.txt) - 500 and 5000 samples of the following 5 random variables:

Column 0: S ... stress (false (0) or true (1))     
Column 1: E ... easily catches cold (false (0) or true (1))  
Column 2: G ... genetic disposition (false (0) or true (1))   
Column 3: I ... increased blood pressure (false (0) or true (1))   
Column 4: H ... heart attack (false (0) or true (1))   

In [148]:
# load the training dataset Y. Note: the file "train.txt" has to be in the same directory you started the ipython notebook server in
Y = np.loadtxt('train.txt', dtype=int)

# load the training dataset Z (used for last exercise). Again, the file has to be in correct directory.
Z = np.loadtxt('test.txt', dtype=int)

# row indices of the random variables
_s_, _e_, _g_, _i_, _h_ = 0, 1, 2, 3, 4

na = newaxis

#print Z.shape[0]
#print Z.shape[1]
indices = [[0], [1, 0], [2], [3, 2, 0], [4, 3]]
a = [0] * size(indices)
b = [0] * size(indices)
print size(b)
i = 0
for c in indices:
    a[i] = c[0]
    b[i] = c[1:] or []
    i += 1
    
print a[0]
print b
print range(5)
print range(5)[::-1]
print type(range(5))
print list(range(5))

#j = 0
#for d in range(5):
for idx,d in enumerate(range(5)):
    indices[idx] = [d]
    #j += 1
    
print indices
#indices2 = [[0], [1, 0], [2], [3, 2, 0], [4, 3]]
#indices2[4].append(2)
#print indices2
#l = [0, 1, 2]
#print indices2[2][0]
l = [0, 1, 2, 3, 4]

for c in it.combinations(l, 4):
    print c

5
0
[[], [0], [], [2, 0], [3]]
[0, 1, 2, 3, 4]
[4, 3, 2, 1, 0]
<type 'list'>
[0, 1, 2, 3, 4]
[[0], [1], [2], [3], [4]]
(0, 1, 2, 3)
(0, 1, 2, 4)
(0, 1, 3, 4)
(0, 2, 3, 4)
(1, 2, 3, 4)


# Helper functions for probability tables

In [29]:
def mle_1d(X, x):
    """ 
    Probability distribution of a variable: P(VAR)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param x: column of the X matrix of which we want the probability distribution
    :returns: array with the probability distribution of the required variable, given the array
    """
    sm = X[:, x].sum()
    num = X[:, x].shape[0] 
    return array([1 - sm / float(num), sm / float(num)])
    
def mle_2d(X, a, b):
    """ 
    Conditional probability distribution of a variable, given another variable: P(VAR1 | VAR2)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param a: column of the X matrix of which we want the conditional probability distribution
    :param b: conditioning column of the X matrix
    :returns: array with the probability distribution of the required variable, given the array and the conditioning variable
    """
    x = X[:, a]
    y = X[:, b]
    num = X.shape[0] 
    condA = where((x == 0) & (y == 0))[0].shape[0] / float(num)
    condB = where((x == 0) & (y == 1))[0].shape[0] / float(num)
    condC = where((x == 1) & (y == 0))[0].shape[0] / float(num)
    condD = 1 - (condA + condB + condC)   
    ab = array([[condA, condB], [condC, condD]])
    return ab / ab.sum(1) / (ab / ab.sum(1)).sum(0)
    

def mle_3d(X, a, b, c):  
    """ 
    Conditional probability distribution of a variable, given two other variables: P(VAR1 | VAR2, VAR3)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param a: column of the X matrix of which we want the conditional probability distribution
    :param b: first conditioning column of the X matrix
    :param c: second conditioning column of the X matrix
    :returns: array with the probability distribution of the required variable, given the array and the conditioning variables
    """    
    x = X[:, a]
    y = X[:, b]
    z = X[:, c]
    num = X.shape[0] 
    condA = where((x == 0) & (y == 0) & (z == 0))[0].shape[0] / float(num)
    condB = where((x == 0) & (y == 0) & (z == 1))[0].shape[0] / float(num)
    condC = where((x == 0) & (y == 1) & (z == 0))[0].shape[0] / float(num)
    condD = where((x == 0) & (y == 1) & (z == 1))[0].shape[0] / float(num)
    condE = where((x == 1) & (y == 0) & (z == 0))[0].shape[0] / float(num)
    condF = where((x == 1) & (y == 0) & (z == 1))[0].shape[0] / float(num)
    condG = where((x == 1) & (y == 1) & (z == 0))[0].shape[0] / float(num)
    condH = 1 - (condA + condB + condC + condD + condE + condF + condG)  
    abc =  array([[[condA, condB], [condC, condD]], [[condE, condF], [condG, condH]]])   
    bc = abc.sum(0)
    return abc / bc

In [141]:
def mle(data, var, parents=None):
    """
    Does maximum likelihood estimation of the (conditional) probability distribution
    of a random variable given the data.
    
    :param data:    data to use for estimating the distribution
    :param var:     index of the variable to estimate the distribution for
    :param parents: parents (ids of given variables) of the variable in the Bayes net,
                    given as list. If None or of length 0, the variable is assumed to have no parents.
    :return:        Estimated (conditional) probability distribution of the random variable
    """
    parents = parents or []
    n_vars = len(parents) + 1
    
    Y = data[:, [var] + parents]
    X = empty((2,) * n_vars)
    
    for vals in it.product([0, 1], repeat=n_vars):
        X[vals] = np.sum(np.all(Y == vals, axis=1))
        
    return X / X.sum(0)


def log_likelihood(data, dists, indices):
    """
    Computes the log likelihood of the data given a model. A model is given as a list
    of distributions (conditional probability tables) and corresponding indices describing
    which dimension of each distribution corresponds to which random variable. E.g., 
    assume you have data for three random variables A, B and C given in alphabetical order.
    If dists looks like [A_B, B, C_AB], indices would be [(0, 1), (1,), (2, 0, 1)].
    
    :param dists:   list of (conditional) probability tables for the random variables
    :param indices: list of variable indices corresponding to the CPTs stored in 'dists'
    :param data:    data to compute the log-likelihood of
    :return:        log-likelihood of data given the model as encoded by dists and indices
    """
    log_l = 0
    for y in data:
        for i in range(len(dists)):
            idx = indices[i]
            log_l += np.log(dists[i][tuple(y[idx])])
            
    return log_l


def mean_log_likelihood(data, dists, indices):
    """
    Computes the mean log likelihood of each data point in data.
    :param dists:   list of (conditional) probability tables for the random variables
    :param indices: list of variable indices corresponding to the CPTs stored in 'dists'
    :param data:    data to compute the log-likelihood of
    :return:        mean log-likelihood of data given the model as encoded by dists and indices
    """
    return log_likelihood(data, dists, indices) / len(data)

def bic(data, dists, indices):
    """
    Computes the Bayesian information score
    :param dists:   list of (conditional) probability tables for the random variables
    :param indices: list of variable indices corresponding to the CPTs stored in 'dists'
    :param data:    data to compute the log-likelihood of
    :return:        Bayesian information score
    """
    i = 0
    for c in indices:
        if len(c) == 1:
            i+= 1

    return log_likelihood(data, dists, indices) - 0.5 * log(len(data)) * i

def testScore(data, var_ord, score):
    j = 0
    for d in var_ord:
        indices[j] = [d]
        j += 1
        
    dists = learn_dists(data, indices)    
    return score(data, dists, indices)


In [146]:
def learn_dists(data, indices):
    a = [0] * size(indices)
    b = [0] * size(indices)

    for idx,c in enumerate(indices):
        a[idx] = c[0]
        b[idx] = c[1:] or []
        
    return [mle(data, a[0], b[0]), mle(data, a[1], b[1]), mle(data, a[2], b[2]), mle(data, a[3], b[3]), mle(data, a[4], b[4])]    
    

# 1.1.A) Probability tables
of P(S), P(G), P(I), P(E), P(H).

In [143]:
S = mle_1d(Y,_s_)
G = mle_1d(Y,_g_)
I = mle_1d(Y,_i_)
E = mle_1d(Y,_e_)
H = mle_1d(Y,_h_)
testScore(Y, score=bic, var_ord=range(5))

-986.34842317179766

# 1.1.B) log likelihood of model 1 relative to the train dataset.

In [147]:
SEGIH1 = S[:, na, na, na, na] * E[na, :, na, na, na] * G[na, na, :, na, na] * I[na, na, na, :, na] * H[na, na, na, na, :] 
ll1r = log(SEGIH1[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 1 relative to the train dataset is ' + `ll1r` + ' .'
dists1 = [mle(Y, 0), mle(Y, 1), mle(Y, 2), mle(Y, 3) , mle(Y, 4)]
indices1 = [[0], [1], [2], [3], [4]]
print log_likelihood(Y, dists1, indices1)
print mean_log_likelihood(Y, dists1, indices1)
print bic(Y, dists1, indices1)
print testScore(Y, indices1, score = bic)

The log likelihood of model 1 relative to the train dataset is -970.81190292576616 .
-970.811902926
-1.94162380585
-986.348423172
-986.348423172


# 1.2.A) Probability tables
of P(E|S), P (I | G, S), and P (H | I) (The remaining ones are already given by 1.1.A)).

In [18]:
E_S = mle_2d(Y, _e_, _s_)
H_I = mle_2d(Y, _h_, _i_)
I_GS = mle_3d(Y, _i_, _g_, _s_)

# 1.2.B) log likelihood of model 2 relative to the train dataset

In [126]:
SEGIH2 = S[:, na, na, na, na] * E_S.transpose((1, 0))[:, :, na, na, na] * G[na, na, :, na, na] * I_GS.transpose((2, 1, 0))[:, na, :, :, na] * H_I.transpose((1, 0))[na, na, na, :, :] 
ll2r = log(SEGIH2[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 2 relative to the train dataset is ' + `ll2r` + ' .'
#dists2 = [mle(Y, 0), mle(Y, 1, [0]), mle(Y, 2), mle(Y, 3, [2, 0]), mle(Y, 4, [3])]
indices2 = [[0], [1, 0], [2], [3, 2, 0], [4, 3]]


    
dists = learn_dists(Y, indices2)
print log_likelihood(Y, dists2, indices2)
print mean_log_likelihood(Y, dists2, indices2)
print bic(Y, dists2, indices2)

The log likelihood of model 2 relative to the train dataset is -940.76297860888167 .
-940.762978609
-1.88152595722
-946.977586707


# 1.3.A) Probability tables
of P (S | G), P (E | S, I), and P (H | I, E) (The remaining ones are already given by 1.1.A) and 1.2.A)).


In [8]:
S_G = mle_2d(Y, _s_, _g_)
E_SI = mle_3d(Y, _e_, _s_, _i_)
H_IE = mle_3d(Y, _h_, _i_, _e_)

# 1.3.B) log likelihood of model 3 relative to the train dataset

In [52]:
SEGIH3 = S_G[:, na, :, na, na] * E_SI.transpose((1, 0, 2))[:, :, na, :, na] * G[na, na, :, na, na] * I_GS.transpose((2, 1, 0))[:, na, :, :, na] * H_IE.transpose((2, 1, 0))[na, :, na, :, :] 
ll3r = log(SEGIH3[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 3 relative to the train dataset is ' + `ll3r` + ' .'
#dists3 = [G, S_G, E_SI, I_GS, H_IE]
dists3 = [mle(Y, 2), mle(Y, 0, [2]), mle(Y, 1, [0, 3]), mle(Y, 3, [2, 0]), mle(Y, 4, [3, 1])]
indices3 = [[2], [0, 2], [1, 0, 3], [3, 2, 0], [4, 3, 1]]
print log_likelihood(Y, dists3, indices3)
print mean_log_likelihood(Y, dists3, indices3)
print bic(Y, dists3, indices3)

The log likelihood of model 3 relative to the train dataset is -939.73403209446235 .
-939.734032094
-1.87946806419
1
-942.841336144


# 2.A) Compare training data
We can state that minimizing the negative log likelihood is equivalent to maximize the likelihood estimation (MLE).  
The model that maximizes the likelihood is clearly the best.

In [10]:
trainll = abs(array([ll1r, ll2r, ll3r]))
argmin(trainll)
min(trainll)
print 'The best model for the training data is ' + `argmin(trainll) + 1` + ', '
print 'because is the smallest, with a negative log likelihood of ' + `min(trainll)` + ' .'

The best model for the training data is 3, 
because is the smallest, with a negative log likelihood of 939.73403209446235 .


The third model, the most complex of the three models, with a small training set, looks like to be the most effective.  
The second model has a log likelihood very close to the third, leaving us wondering if all that complexity was really necessary.  
The first model is the worst by far, probably due to ist excessive simplicity.

# 2.B) Compare test data

In [11]:
testll = abs(array([log(SEGIH1[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum(), 
                    log(SEGIH2[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum(),
                    log(SEGIH3[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum()]))
print testll
argmin(testll)
min(testll)
print 'The best model for the test data is ' + `argmin(testll) + 1` + ', '
print 'because is the smallest, with a negative log likelihood of ' + `min(testll)` + ' .'

[ 9545.45920494  9216.89949044  9226.91241436]
The best model for the test data is 2, 
because is the smallest, with a negative log likelihood of 9216.89949044479 .


The second model, the most reasonable of the three models, with a large test set, is the most effective.  
The third model, as feared, performs worse than the second, probably because of overfitting.  
The first model is again the worst by far, again probably due to ist excessive simplicity.

*Given the reasons in 3.A) and 3.B), I would choose the second model as the best for heart diseases.*

In [54]:
print mean_log_likelihood(Z, dists1, indices1)
print bic(Z, dists1, indices1)
print mean_log_likelihood(Z, dists3, indices3)
print bic(Z, dists2, indices2)
print mean_log_likelihood(Z, dists2, indices2)
print bic(Z, dists3, indices3)

-1.90909184099
5
-9566.75218792
-1.84538248287
2
-9225.41668364
-1.84337989809
1
-9231.17101095
